In [11]:
# http://resource.data.one.gov.hk/td/carpark/basic_info_all.json
# http://resource.data.one.gov.hk/td/carpark/basic_info_tdcp1.json
# https://resource.data.one.gov.hk/td/carpark/vacancy_all.json

import json
import requests

# download the json file
url = "http://resource.data.one.gov.hk/td/carpark/basic_info_all.json"
response = requests.get(url)
data = response.json()
json.dump(data, open("carpark.json", "w", encoding="utf-8"), ensure_ascii=False, indent=4)

url = "https://resource.data.one.gov.hk/td/carpark/vacancy_all.json"
response = requests.get(url)
data = response.json()
json.dump(data, open("vacancy.json", "w", encoding="utf-8"), ensure_ascii=False, indent=4)

In [ ]:
# parse the json file
carpark = json.load(open("carpark.json", "r", encoding="utf-8"))
carpark

{'car_park': [{'park_id': 'tdc166p1',
   'name_en': 'Phase 1 Carpark of Tin Ching Estate',
   'name_tc': '天晴邨第一期停車場',
   'name_sc': '天晴邨第一期停车场',
   'displayAddress_en': 'Tin Wah RD, Aera 103, Tin Shui Wai, N.T.',
   'displayAddress_tc': '新界天水圍103區天華路',
   'displayAddress_sc': '新界天水围103区天华路',
   'latitude': 22.464178165652,
   'longitude': 114.00256679813,
   'district_en': 'Yuen Long',
   'district_tc': '元朗區',
   'district_sc': '元朗区',
   'contactNo': '34018578',
   'opening_status': None,
   'height': 0,
   'remark_en': 'Height Limit: \n',
   'remark_tc': '高度限制: \n',
   'remark_sc': '高度限制: \n',
   'website_en': '',
   'website_tc': '',
   'website_sc': '',
   'carpark_photo': 'http://resource.data.one.gov.hk/td/carpark/tdc166p1.jpg'},
  {'park_id': 'tdc6p11',
   'name_en': 'Cheung Shan Estate Carpark',
   'name_tc': '象山邨停車場',
   'name_sc': '象山邨停车场',
   'displayAddress_en': 'Cheung Shan Estate Road West, Tsuen Wan, N.T.',
   'displayAddress_tc': '新界荃灣象山邨西路',
   'displayAddress_sc': '新界荃